This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-architecture-publish-cv-resnet-model).

## Run Anywhere for ARM Architecture Tutorial: Custom Inference Computer Vision with Resnet50

Wallaroo Run Anywhere provides model deployment in any device, any cloud, and any architecture.  Models uploaded to Wallaroo are set to their targeted architecture.  By default, this is `X86`.

The model's deployment configuration is tied to its architecture settings.  That deployment configuration is carried over to the models publication in Open Container Initiative (OCI) Registries, which allows edge model deployments on X64 and ARM architectures.

This tutorial demonstrates deploying a computer vision model trained to detect objects using the Resnet50 methodology, combined with custom inference known as [Wallaroo Bring Your Own Predict or Arbitrary Python](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) to X64 and ARM edge locations through the following steps.

* Setting up a workspace, and pipeline.
* Upload a model set to the ARM architecture.
* Deploy the model to the ARM architecture and demonstrate that the deployment configuration inherits its architecture from the model configuration.
* Performing a sample set of inferences to verify the deployment.
* Publish the deployed model to an Open Container Initiative (OCI) Registry ARM deployments and verify it inherits the edge deployment architecture based on the model configuration.
* Deploy the model to an ARM edge device.
* Perform similar inferences on the edge device and show the results.

## Goal

Demonstrate models uploaded to Wallaroo with the targeted architecture set to ARM are deployable on ARM architecture nodepools and ARM edge devices.

### Resources

This tutorial provides the following:

* Models:
  * `model-with-pixel-intensity.zip`: A custom inference model combined with the Resnet50 model.  This is downloaded from the Wallaroo Public Data bucket from the following URL.  Unzip the following .zip file into the `./models` folder:
    * [https://storage.googleapis.com/wallaroo-public-data/cv-demo-models/cv-retail-models.zip](https://storage.googleapis.com/wallaroo-public-data/cv-demo-models/cv-retail-models.zip)
  * Various inputs and helper utilities:
    * `./utils.py`: A utility library for converting images into tensor values that are used with pandas DataFrames for inference requests.
    * `./data/images/example`: A set of example images.

### Prerequisites

* If using this tutorial outside of the JupyterHub service integrated into a Wallaroo installation by using the [Wallaroo SDK], the following libraries must be included.  See the `./requirements.txt` for a complete list.
  * onnx==1.12.0
  * onnxruntime==1.12.1
  * torchvision
  * torch
  * matplotlib==3.5.0
  * opencv-python
  * imutils
  * pytz
  * ipywidgets
* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * `json`: Used for format input data for inference requests.
* An ARM Docker deployment environment to deploy the model on an edge location.


## Steps

* Upload the model with the targeted architecture set to `ARM`.
* Create the pipeline add the model as a model step.
* Deploy the model in the targeted architecture and perform sample inferences.
* Publish the pipeline an OCI registry.
* Deploy the model from the pipeline publish to the edge deployment with ARM architecture.
* Perform sample inferences on the ARM edge model deployment.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [25]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from wallaroo.engine_config import Architecture

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
# used to set the model's input/output schemas
import pyarrow as pa

import utils
import datetime
import time

import requests

workspace_name = f'run-anywhere-architecture-demonstration-cv-byop-tutorial'
arm_pipeline_name = f'architecture-demonstration-arm'
model_name_arm = f'computer-vision-with-pixel-intensity-arm'
model_file_name = './models/model-with-pixel-intensity.zip'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [4]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [6]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace names are unique across the Wallaroo instance.  Verify either that another workspace with the same name does not exist, or that access is granted to the user.

In [7]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'run-anywhere-architecture-demonstration-cv-byop-tutorial', 'id': 63, 'archived': False, 'created_by': 'b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', 'created_at': '2024-03-05T15:17:38.68486+00:00', 'models': [], 'pipelines': []}

## Model Deployment on ARM Architecture via the Wallaroo SDK

### Upload Models and Set ARM Target Architecture

For our example, we will upload the model from the file `./data/model-with-pizel-intensity.zip` with the model name `computer-vision-with-pixel-intensity-arm`.

Models are uploaded to Wallaroo via the [`wallaroo.client.upload_model`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/client/#Client.upload_model) method which takes the following arguments:

| Parameter | Type | Description |
|---|---|---|
| **path** | *String* (*Required*) | The file path to the model. |
| **framework** | *wallaroo.framework.Framework* (*Required*) | The model's framework.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for supported model frameworks. |
| **input_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's input schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **output_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's output schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **convert_wait** | *bool* (*Optional*)  | Whether to wait in the SDK session to complete the auto-packaging process for non-native Wallaroo frameworks. |
| **arch** | *wallaroo.engine_config.Architecture* (*Optional*)  | The targeted architecture for the model.  Options are <ol><li>`X86` (*Default*)</li><li>`ARM`</li></ol> |

For this demonstration we set the architecture for the model to `wallaroo.Architecture.ARM`.

In [27]:
input_schema = pa.schema([
    pa.field('tensor', pa.list_(
        pa.list_(
            pa.list_(
                pa.float32(), # images are normalized
                list_size=640
            ),
            list_size=480
        ),
        list_size=3
    )),
])

output_schema = pa.schema([
    pa.field('boxes', pa.list_(pa.list_(pa.float32(), list_size=4))),
    pa.field('classes', pa.list_(pa.int64())),
    pa.field('confidences', pa.list_(pa.float32())),
    pa.field('avg_px_intensity', pa.list_(pa.float32())),
    pa.field('avg_confidence', pa.list_(pa.float32())),
])


housing_model_control_arm = wl.upload_model(model_name_arm, 
                        model_file_name, 
                        framework=Framework.CUSTOM,
                        input_schema=input_schema, 
                        output_schema=output_schema,
                        arch=Architecture.ARM)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime...............successful

Ready


In [28]:
display(housing_model_control_arm)

Name,computer-vision-with-pixel-intensity-arm
Version,23f63ab7-7c17-45bb-af0a-869ba6a7485f
File Name,model-with-pixel-intensity.zip
SHA,6d58039b1a02c5cce85646292965d29056deabdfcc6b18c34adf566922c212b0
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4609
Architecture,None
Acceleration,None
Updated At,2024-05-Mar 15:27:51


### Deploy Models to ARM Architecture

#### Build the Pipeline

This pipeline is used as an example of the edge deployment environment for testing.  We will create a pipeline and add our model set to the `ARM` architecture as a pipeline step.

The **model's targeted architecture** is inherited by the pipeline version; no additional pipeline settings are required to set that architecture.  The other settings apply the CPU and memory used by the model.

In [29]:
pipeline_arm = wl.build_pipeline(arm_pipeline_name)

# clear the previous steps
pipeline_arm.clear()

# set the model step with the ARM targeted model
pipeline_arm.add_model_step(housing_model_control_arm)

#minimum deployment config
deployment_config = wallaroo.DeploymentConfigBuilder() \
    .replica_count(1) \
    .cpus(1) \
    .memory("2Gi") \
    .sidekick_cpus(housing_model_control_arm, 1) \
    .sidekick_memory(housing_model_control_arm, '6Gi') \
    .build()

#### Deploy Pipeline

We deploy the pipeline, then show that the deployment configuration architecture is set to `ARM`.

In [40]:
pipeline_arm.deploy(deployment_config=deploy_config)

Waiting for deployment - this will take up to 45s ............................................

WaitForDeployError: Deployment failed. See status for details.
Status: {'status': 'Error', 'details': [], 'engines': [{'ip': '10.124.3.42', 'name': 'engine-77b8cd7674-dcxsj', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': []}, 'model_statuses': {'models': []}}], 'engine_lbs': [{'ip': '10.124.0.138', 'name': 'engine-lb-d7cc8fc9c-6vhlz', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}

In [ ]:
pipeline_arm.status()

### Inferences on ARM Architecture Model Deployments

With the pipeline deployed, we'll perform sample inferences through the pipeline's inference API endpoints.  This will be the same method used for inference requests for model edge deployments.

The following uses the included `utils.py` to convert the image into its tensor values.  It takes in the file name and the sample height and weight which for this is example is set to 640x480, and returns a DataFrame of the converted image, and the resized image.

In [32]:
width, height = 640, 480

sample_image = "./data/images/input/example/dairy_bottles.png"

dfImage, resizedImage = utils.loadImageAndConvertToDataframe(sample_image, width, height)

In [33]:
# get the pipeline deployment urls

arm_deploy_url = pipeline_arm._deployment._url()

# get authorization header
headers = wl.auth.auth_header()

# set the content-type and accept headers

# set the content type for pandas records
headers['Content-Type']= 'application/json; format=pandas-records'

# set accept as pandas-records
headers['Accept']='application/json; format=pandas-records'

In [36]:
### inferences on arm model deployments


headers = {
        'Content-Type': 'application/json; format=pandas-records'
    }

# retrieve response
response = requests.post(
                    arm_deploy_url, 
                    headers=headers, 
                    data=dfImage.to_json(orient="records")
                )
response.json()
# convert response to dataframe

inference_result = pd.DataFrame(response.json())

{'status': 'Error',
 'error': 'Pipeline with id architecture-demonstration-arm not found'}

The inference results are displayed.  The bounding boxes of detected objects are displayed with the `utils.drawDetectedObjectsFromInference` method, and the `out.avg_confidence` field displayed directly after.

In [ ]:
image = utils.drawDetectedObjectsFromInference(inference_result)
    
display(inference_result.loc[:, ['out.avg_confidence']])

### Undeploy the Pipelines

With the inference examples complete, we can undeploy the pipeline.

In [ ]:
pipeline_arm.undeploy()

## Edge Deployment

We now deploy the pipeline versions to our edge devices.

* Publish the pipeline versions:  Publishes the pipeline versions to the OCI registry with the deployment configuration inherited from the pipeline versions, with the architecture settings inherited from the model versions.
* Deploy Edge:  Deploy the edge device with the edge location settings.

### Publish Pipeline Versions

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [ ]:
assay_pub_arm = pipeline_arm.publish()
assay_pub_arm

### DevOps Deployment on ARM Architecture

The edge deployment is performed with `docker run`, `docker compose`, or `helm` installations.  The following command generates the `docker run` command, with the following values provided by the DevOps Engineer:

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

In [ ]:
# create docker run 

docker_command = f'''
docker run -p 8080:8080 \\
    -e DEBUG=true \\
    -e OCI_REGISTRY=$REGISTRYURL \\
    -e CONFIG_CPUS=1 \\
    -e OCI_USERNAME=$OCI_USERNAME \\
    -e OCI_PASSWORD=$OCI_PASSWORD \\
    -e PIPELINE_URL={assay_pub_arm.pipeline_url} \\
    {assay_pub_arm.engine_url}
'''

print(docker_command)

### Edge Inference Examples on ARM Architecture

The following examples demonstrate performing inferences on the model deployed on an `ARM` architecture edge device.  `HOSTNAME_ARM` is the hostname for the ARM architecture edge deployment.  Update this variable to match your deployment.

In [ ]:
HOSTNAME_ARM = 'localhost'

In [ ]:
# perform the inference through the ARM edge device

edge_inference_url = f'{HOSTNAME_ARM}:8080/pipelines/{arm_pipeline_name}'

headers = {
        'Content-Type': 'application/json; format=pandas-records'
    }

# retrieve response
response = requests.post(
                    edge_inference_url, 
                    headers=headers, 
                    data=dfImage.to_json(orient="records")
                )
response.json()
# convert response to DataFrame and display

inference_result = pd.DataFrame(response.json())
image = utils.drawDetectedObjectsFromInference(inference_result)
    
display(inference_result.loc[:, ['out.avg_confidence']])